In [1]:
# import libraries
import re
import urllib3
import certifi

In [2]:
# constract all regular expressions ( I added regex in a text separated file for better reading access chek it if needed)
title_regex=re.compile(r"class=\"title\">(.+)")
date_regex=re.compile(r"class=\"clamp-details\">\s+(.+)")
description_regex=re.compile(r"\"summary\">\s*([\S\s]+?)\s*")
score_regex = re.compile(r"\"title\">Metascore:\s+\"metascore_anchor\" href=\"/movie/.*?/critic-reviews\">\s+\"metascore_w large movie .+\">(.*?)")
image_regex = re.compile(r"\"/movie/.*\">\"(.*)\" alt=\"")

# Construct an HTTP pool for connection
http=urllib3.PoolManager(ca_certs=certifi.where())

In [3]:
import json
import pymongo
import certifi
# retrieve creditials to connect MongoDB
with open('F:/secrets.json') as f:
    data = json.load(f)
    secret_key = data['mongodb']

# Connect to the database using known good certificates
client = pymongo.MongoClient(secret_key, tlsCAFile=certifi.where())
# Fetch the database named "DA-320"
da320_database = client['IMDB_METACRITIC']

# Within the database we have "collections". Think of them as tables in SQL.
metacritic_data = da320_database['metacritic']

In [4]:
import pandas as pd
#retrive a list of movies from a particular year and page of Metacritic
def metacritic_scraper (year:int,page:int) -> pd.DataFrame:
    #fetch the webpage
   url = f"https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected={year}&sort=desc&view=detailed&page={page}"
   # Construct python request pool
   response = http.request('GET',url,headers={'User-Agent':'Mozilla/5.0'})
   datastring = str(response.data, "utf-8")

   #execute all regular expression
   titles = title_regex.findall(datastring)
   dates = date_regex.findall(datastring)
   descriptions = description_regex.findall(datastring)
   scores = score_regex.findall(datastring)
   images = image_regex.findall(datastring)
   #return unified collection  
   print("title: " , len(titles), "date: " , len(dates), "description: " , len(descriptions), "score: " , len(scores), "image: " , len(images))
   dataset = {"title": titles, "date": dates, "description": descriptions, "score": scores, "image":images}
   return pd.DataFrame(dataset) 

In [ ]:
import re
import time

#Writing to CVS file
for year in range(2000,2023):
   page=0
   print(f"Collection date for {year} page {page}.....")

   #retry a page multiple times
   while True:
      data = metacritic_scraper(year,page)

      #stop when we reach a page with zero rows
      if len(data) == 0:
         break

      #conver the dataframe into a list of movies to insert into Mongo DB
      movies_to_insert =[]
      for row in data.itertuples():
         movie = {
            "title": row.title,
            "release_date": row.date,
            "description": row.description,
            "metascore": row.score,
            "image_url": row.image,
         }
         movies_to_insert.append(movie)
         
      #insert resords into MongoDB
      print(f"Inserting{len(movies_to_insert)} moview for year {year} page {page}")
      metacritic_data.insert_many(movies_to_insert)
      page = page+1